# 1- Machine Learning: Introdução a classificação

Não executar celular individualmente...necessário reiniciar todo o programa para evitar troca de valores entre variáveis com o mesmo nome em células diferentes

## Bibliotecas

In [1]:
#bibliotecas
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier
from collections import Counter
import csv
import pandas as pd
import numpy as np
import nltk
#nltk.download('stopwords')

## Introdução ao MultinomialNB

In [2]:
#Array de caracteristicas [gordo, pernas curtas, late]. 1 - Sim, 0 - Não
porco1 = [1,1,0]
porco2 = [1,0,1]
porco3 = [0,1,0]

cachorro1 = [1,1,1]
cachorro2 = [0,1,1]
cachorro3 = [0,1,1]

dados = [porco1, porco2, porco3, cachorro1, cachorro2, cachorro3]
marcações = [1,1,1,-1,-1,-1] # 1 - porco, -1 - cachorro

#Dados para teste
misterioso1 = [1,1,1]
misterioso2 = [1,0,0]
misterioso3 = [1,0,1]
misterioso4 = [0,1,1]
misteriosos = [misterioso1, misterioso2, misterioso3, misterioso4]

#Treinamento do modelo
modelo = MultinomialNB()
modelo.fit(dados, marcações)

#Prever para array 'misterioso'
resultado = modelo.predict(misteriosos)
print(resultado)

#Estudo de margem de erro. Vamos agora comparar o resultado do nosso modelo para o array de testes com o real
#resultado armagenado no array 'marcações_misterioso'

#Resultado esperando para array 'misterioso':
marcações_misterioso = [-1,1,-1,-1]

#Se os resultados forem iguais o elemento no array será 0
comp = marcações_misterioso - resultado 
acerto = [acertos for acertos in comp if acertos == 0]

k, n = len(acerto), len(comp)
print("Taxa de acerto é de", k/n*100, "%")

[ 1  1  1 -1]
Taxa de acerto é de 50.0 %


## Primeiro caso real

In [3]:
def carregar_acessos():
    X = []
    Y = []
    arquivo = open('acesso_pagina.csv', 'r')
    leitor = csv.reader(arquivo)
    next(leitor)
    
    for acessou_home,acessou_como_funciona,acesso_contato,comprou in leitor:
        X.append([int(acessou_home), int(acessou_como_funciona), int(acesso_contato)])
        Y.append([int(comprou)])
        
    return X, Y

X, Y = carregar_acessos()
Y = [item for sublist in Y for item in sublist]

modeloAlura = MultinomialNB()
modeloAlura.fit(X, Y)

teste = [[1,0,1], [0,1,0], [1,0,0], [1,1,0], [1,1,1]]

resultado = modeloAlura.predict(X)
comparacao = resultado - Y

acertos = [erro for erro in comparacao if erro == 0]

taxa = len(acertos)/len(X)
print("Taxa de acerto é",taxa*100,"%")

Taxa de acerto é 93.93939393939394 %


## Sobre utilizar os dados do treino nos testes

In [35]:
#No exemplo anterior testamos o modelo com os mesmos dados
#que usamos para treinar...isso nos entrega uma taxa de
#acerto falsa. Precisamos encontrar uma forma de saber,
#de todos os nossos dados, quantos usar para treinar e 
#quantos usar para testar.

def carregar_acessos():
    X = []
    Y = []
    arquivo = open('acesso_pagina.csv', 'r')
    leitor = csv.reader(arquivo)
    next(leitor)
    
    for acessou_home,acessou_como_funciona,acesso_contato,comprou in leitor:
        X.append([int(acessou_home), int(acessou_como_funciona), int(acesso_contato)])
        Y.append([int(comprou)])
        
    return X, Y

X, Y = carregar_acessos()
Y = [item for sublist in Y for item in sublist]

treino_dados = X[:90]
treino_marcacoes = Y[:90]

teste_dados = X[-9:]
teste_marcacoes = Y[-9:]

modeloAlura = MultinomialNB()
modeloAlura.fit(treino_dados, treino_marcacoes)

resultado = modeloAlura.predict(teste_dados)
comparacao = resultado - teste_marcacoes

acertos = [erro for erro in comparacao if erro == 0]

taxa = len(acertos)/len(teste_dados)
print("Taxa de acerto é",taxa*100,"%")

Taxa de acerto é 88.88888888888889 %


## Utilizando DataFrame e Caracteristicas categóricas

In [36]:
#Vamos usar o pandas para ler os dados muito mais facil-
#mente. O pandas consegue fazer todo o papel da função que
#que haviamos criado sem se precisar com tipos de dados

df = pd.read_csv('dados_cat.csv')
X_df, Y_df = df[['home','busca','logado']], df['comprou']

#Vamos transformar a coluna categorica 'algoritmo' em 3
#colunas binárias
X_df = pd.get_dummies(X_df)
 
X = X_df.values
Y = Y_df.values

n = int(0.9 * len(X))
m = int(len(X) - n)

treino_dados     = X[:n] 
treino_marcacoes = Y[:n]
teste_dados      = X[-m:]
teste_marcacoes  = Y[-m:]

modelo = MultinomialNB()
modelo.fit(treino_dados,treino_marcacoes)

resultados = modelo.predict(teste_dados)
comparacao = resultados - teste_marcacoes

acertos = [acerto for acerto in comparacao if acerto == 0]

print("A taxa de acerto é",len(acertos)/len(comparacao)*100,"%")

#Mas essa taxa de acerto é um valor bom? Como saber? Vamos fazer um algoritmo burro que chuta o mesmo valor para
#todos os testes

acerto_burro = max(len(teste_marcacoes[teste_marcacoes == 1]), len(teste_marcacoes[teste_marcacoes == 0]))
taxa_acerto = 100 * acerto_burro/len(teste_marcacoes)
print("A taxa de acerto base é", taxa_acerto, "%")

#Ou seja, o nosso algoritmo é tão ruim quanto um que chuta o valor que mais aparece nos dados. Precisamos melhorar

A taxa de acerto é 82.0 %
A taxa de acerto base é 82.0 %


## Apresentando AdaBoost

In [37]:
#Vamos testar o modelo usando AdaBoostClassifier e o MultinomialNB para encontrar a melhor 
#versão do nosso modelo com varias combinações de variaveis utilizadas

def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes):
    
    modelo.fit(treino_dados,treino_marcacoes)
    resultados = modelo.predict(teste_dados)
    comparacao = resultados - teste_marcacoes
    acertos = [acerto for acerto in comparacao if acerto == 0]
    taxa_de_acertos = 100*len(acertos)/len(comparacao)
    msg = "A taxa de acerto {0}: {1}".format(nome, taxa_de_acertos)
    print(msg)
    
df = pd.read_csv('dados_cat2.csv')
X_df, Y_df = df[['home','busca','logado']], df['comprou']

X_df = pd.get_dummies(X_df)
 
X = X_df.values
Y = Y_df.values

n = int(0.9 * len(X))
m = int(len(X) - n)

treino_dados     = X[:n] 
treino_marcacoes = Y[:n]
teste_dados      = X[-m:]
teste_marcacoes  = Y[-m:]

modelo = AdaBoostClassifier()
fit_and_predict("AdaBoostClassifier",modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

modelo = MultinomialNB()
fit_and_predict("MultinomialNB", modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

acerto_burro = max(len(teste_marcacoes[teste_marcacoes == 1]), len(teste_marcacoes[teste_marcacoes == 0]))
taxa_acerto = 100 * acerto_burro/len(teste_marcacoes)
print("A taxa de acerto base é", taxa_acerto, "%")

A taxa de acerto AdaBoostClassifier: 75.0
A taxa de acerto MultinomialNB: 75.0
A taxa de acerto base é 62.5 %


## Separando treino, teste e validação

In [38]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes):
    
    modelo.fit(treino_dados,treino_marcacoes)
    resultados = modelo.predict(teste_dados)
    comparacao = resultados - teste_marcacoes
    acertos = [acerto for acerto in comparacao if acerto == 0]
    taxa_de_acertos = 100*len(acertos)/len(comparacao)
    msg = "A taxa de acerto {0}: {1}".format(nome, taxa_de_acertos)
    print(msg)
    return taxa_de_acertos
    
df = pd.read_csv('dados_cat2.csv')
X_df, Y_df = df[['home','busca','logado']], df['comprou']

X_df = pd.get_dummies(X_df)
 
X = X_df.values
Y = Y_df.values

tamanho_treino = int(0.8 * len(Y))
tamanho_teste = int(0.1 * len(Y))
tamanho_validacao = int(tamanho_treino - tamanho_teste)

treino_dados        = X[:tamanho_treino] 
treino_marcacoes    = Y[:tamanho_treino]
teste_dados         = X[tamanho_treino:tamanho_treino + tamanho_teste]
teste_marcacoes     = Y[tamanho_treino:tamanho_treino + tamanho_teste]
validacao_dados     = X[-tamanho_teste:] 
validacao_marcacoes = Y[-tamanho_teste:]

modeloAda = AdaBoostClassifier()
resultado_adaboost = fit_and_predict("AdaBoostClassifier", modeloAda, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

modeloNB = MultinomialNB()
resultado_multinomial = fit_and_predict("MultinomialNB", modeloNB, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

acerto_burro = max(len(teste_marcacoes[teste_marcacoes == 1]), len(teste_marcacoes[teste_marcacoes == 0]))
taxa_acerto = 100 * acerto_burro/len(teste_marcacoes)
print("A taxa de acerto base é", taxa_acerto, "%")

if resultado_adaboost > resultado_multinomial:
    vencedor = modeloAda
else: vencedor = modeloNB
    
resultado_real = vencedor.predict(validacao_dados)

comparacao_real = resultado_real - validacao_marcacoes
acertos_real = [acerto for acerto in comparacao_real if acerto == 0]
taxa_de_acertos_real = 100*len(acertos_real)/len(comparacao_real)
msg = "A taxa de acerto do vencedor {0}: {1}".format("no mundo real", taxa_de_acertos_real)
print(msg)

A taxa de acerto AdaBoostClassifier: 85.71428571428571
A taxa de acerto MultinomialNB: 85.71428571428571
A taxa de acerto base é 85.71428571428571 %
A taxa de acerto do vencedor no mundo real: 85.71428571428571


# 2 - Machine Learning II: Avançando com tipos diferentes de classificação


## Mais algoritimos de classificação

In [39]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes):
    
    modelo.fit(treino_dados,treino_marcacoes)
    resultados = modelo.predict(teste_dados)
    comparacao = resultados - teste_marcacoes
    acertos = [acerto for acerto in comparacao if acerto == 0]
    taxa_de_acertos = 100*len(acertos)/len(comparacao)
    msg = "A taxa de acerto {0}: {1}".format(nome, taxa_de_acertos)
    print(msg)
    return taxa_de_acertos
    
df = pd.read_csv('situacao_do_cliente.csv')
X_df, Y_df = df[['recencia','frequencia','semanas_de_inscricao']], df['situacao']

X_df = pd.get_dummies(X_df)
 
X = X_df.values
Y = Y_df.values

tamanho_treino = int(0.8 * len(Y))
tamanho_teste = int(0.1 * len(Y))
tamanho_validacao = int(tamanho_treino - tamanho_teste)

treino_dados        = X[:tamanho_treino] 
treino_marcacoes    = Y[:tamanho_treino]
teste_dados         = X[tamanho_treino:tamanho_treino + tamanho_teste]
teste_marcacoes     = Y[tamanho_treino:tamanho_treino + tamanho_teste]
validacao_dados     = X[-tamanho_teste:] 
validacao_marcacoes = Y[-tamanho_teste:]

modeloOVO = OneVsOneClassifier(LinearSVC(random_state = 0))
resultado_OVO = fit_and_predict("OneVsOne", modeloOVO, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

modeloOVR = OneVsRestClassifier(LinearSVC(random_state = 0))
resultado_OVR = fit_and_predict("OneVsRest", modeloOVR, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

modeloAda = AdaBoostClassifier()
resultado_adaboost = fit_and_predict("AdaBoostClassifier", modeloAda, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

modeloNB = MultinomialNB()
resultado_multinomial = fit_and_predict("MultinomialNB", modeloNB, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base:", taxa_de_acerto_base, "%")

resultados = {}
resultados[resultado_OVO] = modeloOVO
resultados[resultado_OVR] = modeloOVR
resultados[resultado_adaboost] = modeloAda
resultados[resultado_multinomial] = modeloNB

vencedor = resultados[max(resultados)]

resultado_real = vencedor.predict(validacao_dados)

comparacao_real = resultado_real - validacao_marcacoes
acertos_real = [acerto for acerto in comparacao_real if acerto == 0]
taxa_de_acertos_real = 100*len(acertos_real)/len(comparacao_real)
msg = "A taxa de acerto do vencedor {0}: {1}".format("no mundo real", taxa_de_acertos_real)
print(msg)

A taxa de acerto OneVsOne: 100.0
A taxa de acerto OneVsRest: 90.9090909090909
A taxa de acerto AdaBoostClassifier: 68.18181818181819
A taxa de acerto MultinomialNB: 72.72727272727273
Taxa de acerto base: 81.81818181818181 %
A taxa de acerto do vencedor no mundo real: 100.0


## Utilizando k-fold

In [40]:
def score(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acertos = 100 * np.mean(scores)
    msg = "A taxa de acerto {0}: {1}".format(nome, taxa_de_acertos)
    print(msg)
    return taxa_de_acertos
    
df = pd.read_csv('situacao_do_cliente.csv')
X_df, Y_df = df[['recencia','frequencia','semanas_de_inscricao']], df['situacao']

X_df = pd.get_dummies(X_df)
 
X = X_df.values
Y = Y_df.values

porcentagem = 0.8
tamanho_treino = int(porcentagem * len(Y))

treino_dados        = X[0:tamanho_treino] 
treino_marcacoes    = Y[0:tamanho_treino]

validacao_dados     = X[tamanho_treino:] 
validacao_marcacoes = Y[tamanho_treino:]

modeloOVO = OneVsOneClassifier(LinearSVC(random_state = 0))
resultado_OVO = score("OneVsOne", modeloOVO, treino_dados, treino_marcacoes)

modeloOVR = OneVsRestClassifier(LinearSVC(random_state = 0))
resultado_OVR = score("OneVsRest", modeloOVR, treino_dados, treino_marcacoes)

modeloAda = AdaBoostClassifier()
resultado_adaboost = score("AdaBoostClassifier", modeloAda, treino_dados, treino_marcacoes)

modeloNB = MultinomialNB()
resultado_multinomial = score("MultinomialNB", modeloNB, treino_dados, treino_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base:", taxa_de_acerto_base, "%")

resultados = {}
resultados[resultado_OVO] = modeloOVO
resultados[resultado_OVR] = modeloOVR
resultados[resultado_adaboost] = modeloAda
resultados[resultado_multinomial]= modeloNB

vencedor = resultados[max(resultados)]
vencedor.fit(treino_dados, treino_marcacoes)

resultado_real = vencedor.predict(validacao_dados)
comparacao_real = resultado_real - validacao_marcacoes
acertos_real = [acerto for acerto in comparacao_real if acerto == 0]
taxa_de_acertos_real = 100*len(acertos_real)/len(comparacao_real)
msg = "A taxa de acerto do vencedor {0}: {1}".format("no mundo real", taxa_de_acertos_real)
print(msg)

A taxa de acerto OneVsOne: 99.44444444444444
A taxa de acerto OneVsRest: 92.31381148950808
A taxa de acerto AdaBoostClassifier: 76.29471964224285
A taxa de acerto MultinomialNB: 82.99772101823186
Taxa de acerto base: 75.55555555555556 %
A taxa de acerto do vencedor no mundo real: 100.0


## Criando um dicionário

In [109]:
classificacoes = pd.read_csv("emails.csv")

textos_puros = classificacoes['email']
textosQuebrados = textos_puros.str.lower().str.split(' ')

dicionario = set()
for lista in textosQuebrados:
    dicionario.update(lista)

tuplas = zip(dicionario, range(total_de_palavras))
tradutor = {palavra: indice for palavra,indice in tuplas}

def vetorizar_texto(texto, tradutor):
    vetor = [0] * len(tradutor)
    for palavra in texto:
        if palavra in tradutor:
            vetor[tradutor[palavra]] += 1
    return vetor
        
def score(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acertos = 100 * np.mean(scores)
    msg = "A taxa de acerto {0}: {1}".format(nome, taxa_de_acertos)
    print(msg)
    return taxa_de_acertos

vetores_de_texto = [vetorizar_texto(texto, tradutor) for texto in textosQuebrados]

X, Y = np.array(vetores_de_texto), np.array(classificacoes['classificacao'].tolist())

porcentagem_de_treino = 0.8

tamanho_treino = int(porcentagem_de_treino * len(Y))
tamanho_validacao = len(Y) - tamanho_treino

treino_dados        = X[0:tamanho_treino] 
treino_marcacoes    = Y[0:tamanho_treino]

validacao_dados     = X[tamanho_treino:] 
validacao_marcacoes = Y[tamanho_treino:]

modeloOVO = OneVsOneClassifier(LinearSVC(random_state = 0))
resultado_OVO = score("OneVsOne", modeloOVO, treino_dados, treino_marcacoes)

modeloOVR = OneVsRestClassifier(LinearSVC(random_state = 0))
resultado_OVR = score("OneVsRest", modeloOVR, treino_dados, treino_marcacoes)

modeloAda = AdaBoostClassifier()
resultado_adaboost = score("AdaBoostClassifier", modeloAda, treino_dados, treino_marcacoes)

modeloNB = MultinomialNB()
resultado_multinomial = score("MultinomialNB", modeloNB, treino_dados, treino_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base:", taxa_de_acerto_base, "%")

resultados = {}
resultados[resultado_OVO] = modeloOVO
resultados[resultado_OVR] = modeloOVR
resultados[resultado_adaboost] = modeloAda
resultados[resultado_multinomial]= modeloNB

vencedor = resultados[max(resultados)]
vencedor.fit(treino_dados, treino_marcacoes)

resultado_real = vencedor.predict(validacao_dados)
comparacao_real = resultado_real - validacao_marcacoes
acertos_real = [acerto for acerto in comparacao_real if acerto == 0]
taxa_de_acertos_real = 100*len(acertos_real)/len(comparacao_real)
msg = "A taxa de acerto do vencedor {0}: {1}".format("no mundo real", taxa_de_acertos_real)
print(msg)


A taxa de acerto OneVsOne: 65.66666666666666
A taxa de acerto OneVsRest: 72.33333333333334
A taxa de acerto AdaBoostClassifier: 47.33333333333333
A taxa de acerto MultinomialNB: 71.50000000000001
Taxa de acerto base: 44.44444444444444 %
A taxa de acerto do vencedor no mundo real: 88.88888888888889


## Limpando dados para melhorar eficiência

In [132]:
#PASSOS:
# Transformar em mínusculo 
# Separar palavras e pontuações
# Remover stopwords
# Transformar a palavra somente na sua raiz (sem genero, plural...)
# Remover com menos de 3 caract. (Para remover pontuação)

classificacoes = pd.read_csv("emails.csv")

textos_puros = classificacoes['email']

frase_min = textos_puros.str.lower()
textosQuebrados = [nltk.tokenize.word_tokenize(frase) for frase in frase_min]

stopwords = nltk.corpus.stopwords.words("portuguese")
stemmer = nltk.stem.RSLPStemmer()
dicionario = set()

for lista in textosQuebrados:
    validas = [stemmer.stem(palavra) for palavra in lista if palavra not in stopwords and len(palavra) > 2]
    dicionario.update(validas)

tuplas = zip(dicionario, range(total_de_palavras))
tradutor = {palavra: indice for palavra,indice in tuplas}

def vetorizar_texto(texto, tradutor):
    vetor = [0] * len(tradutor)
    for palavra in texto:
        if len(palavra) > 0 and stemmer.stem(palavra) in tradutor:
            vetor[tradutor[stemmer.stem(palavra)]] += 1
    return vetor
        
def score(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acertos = 100 * np.mean(scores)
    msg = "A taxa de acerto {0}: {1}".format(nome, taxa_de_acertos)
    print(msg)
    return taxa_de_acertos

vetores_de_texto = [vetorizar_texto(texto, tradutor) for texto in textosQuebrados]

X, Y = np.array(vetores_de_texto), np.array(classificacoes['classificacao'].tolist())

porcentagem_de_treino = 0.8

tamanho_treino = int(porcentagem_de_treino * len(Y))
tamanho_validacao = len(Y) - tamanho_treino

treino_dados        = X[0:tamanho_treino] 
treino_marcacoes    = Y[0:tamanho_treino]

validacao_dados     = X[tamanho_treino:] 
validacao_marcacoes = Y[tamanho_treino:]

modeloOVO = OneVsOneClassifier(LinearSVC(random_state = 0))
resultado_OVO = score("OneVsOne", modeloOVO, treino_dados, treino_marcacoes)

modeloOVR = OneVsRestClassifier(LinearSVC(random_state = 0))
resultado_OVR = score("OneVsRest", modeloOVR, treino_dados, treino_marcacoes)

modeloAda = AdaBoostClassifier()
resultado_adaboost = score("AdaBoost", modeloAda, treino_dados, treino_marcacoes)

modeloNB = MultinomialNB()
resultado_multinomial = score("MultinomialNB", modeloNB, treino_dados, treino_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base:", taxa_de_acerto_base, "%")

resultados = {}
resultados[resultado_OVO] = modeloOVO
resultados[resultado_OVR] = modeloOVR
resultados[resultado_adaboost] = modeloAda
resultados[resultado_multinomial]= modeloNB

vencedor = resultados[max(resultados)]
vencedor.fit(treino_dados, treino_marcacoes)

resultado_real = vencedor.predict(validacao_dados)
comparacao_real = resultado_real - validacao_marcacoes
acertos_real = [acerto for acerto in comparacao_real if acerto == 0]
taxa_de_acertos_real = 100*len(acertos_real)/len(comparacao_real)
msg = "A taxa de acerto do vencedor {0}: {1}".format("no mundo real", taxa_de_acertos_real)
print(msg)


A taxa de acerto OneVsOne: 80.16666666666666
A taxa de acerto OneVsRest: 80.16666666666666
A taxa de acerto AdaBoost: 59.33333333333333
A taxa de acerto MultinomialNB: 83.0
Taxa de acerto base: 44.44444444444444 %
A taxa de acerto do vencedor no mundo real: 77.77777777777777
